In [1]:
import polars as pl

In [2]:
data = pl.read_parquet("data/processed.parquet")
data.head()

borough,zip_code,latitude,longitude,on_street_name,cross_street_name,off_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,date,year,month,time,hour,number_of_casualty
str,i64,f64,f64,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,date,i32,i8,time,i8,i64
null,null,null,null,"""WHITESTONE EXPRESSWAY""","""20 AVENUE""",null,2,0,0,0,0,0,2,0,"""Aggressive Driving/Road Rage""","""Unspecified""",null,null,null,"""Sedan""","""Sedan""",null,null,null,2021-09-11,2021,9,02:39:00,2,2
null,null,null,null,"""QUEENSBORO BRIDGE UPPER""",null,null,1,0,0,0,0,0,1,0,"""Pavement Slippery""",null,null,null,null,"""Sedan""",null,null,null,null,2022-03-26,2022,3,11:45:00,11,1
null,null,null,null,"""THROGS NECK BRIDGE""",null,null,0,0,0,0,0,0,0,0,"""Following Too Closely""","""Unspecified""",null,null,null,"""Sedan""","""Pick-up Truck""",null,null,null,2022-06-29,2022,6,06:55:00,6,0
"""BROOKLYN""",11208,40.667202,-73.8665,null,null,"""1211 LORING AVENUE""",0,0,0,0,0,0,0,0,"""Unspecified""",null,null,null,null,"""Sedan""",null,null,null,null,2021-09-11,2021,9,09:35:00,9,0
"""BROOKLYN""",11233,40.683304,-73.917274,"""SARATOGA AVENUE""","""DECATUR STREET""",null,0,0,0,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,2021-12-14,2021,12,08:13:00,8,0


In [3]:
safety_columns = ["year", "month", "hour", "borough"]
safety_statuses = ["number_of_persons_killed", "number_of_persons_injured"]

status_readable_names = {
    "number_of_persons_killed": "death",
    "number_of_persons_injured": "injury",
    "number_of_casualty": "casualty",
}

safety_data = data.select(safety_columns + safety_statuses).drop_nulls()
# data = data.cast({safety_status: pl.Int64 for safety_status in safety_statuses})

safety_data = safety_data.with_columns(pl.sum_horizontal(safety_statuses).alias("number_of_casualty"))
safety_statuses.append("number_of_casualty")


for column in safety_columns[:-1]:
    joined = pl.DataFrame()

    for i, safety_status in enumerate(safety_statuses):
        columns = ["borough", column]
        pivot = safety_data.pivot(on=columns, index=columns, values=[safety_status], aggregate_function="sum")
        pivot = pivot.with_columns(pl.coalesce(pl.col(pivot.columns[2:]).alias(safety_status)))
        pivot = pivot.select(columns + [safety_status])
        if joined.shape[0] == 0 and joined.shape[1] == 0:
            joined = pivot
        else:
            joined = joined.join(pivot, on=columns)
    print(joined.head())
    joined.write_parquet(f"data/borough_{column}.parquet")

shape: (5, 5)
┌───────────────┬──────┬──────────────────────────┬───────────────────────────┬────────────────────┐
│ borough       ┆ year ┆ number_of_persons_killed ┆ number_of_persons_injured ┆ number_of_casualty │
│ ---           ┆ ---  ┆ ---                      ┆ ---                       ┆ ---                │
│ str           ┆ i32  ┆ i64                      ┆ i64                       ┆ i64                │
╞═══════════════╪══════╪══════════════════════════╪═══════════════════════════╪════════════════════╡
│ BROOKLYN      ┆ 2021 ┆ 54                       ┆ 11676                     ┆ 11730              │
│ BRONX         ┆ 2021 ┆ 23                       ┆ 5757                      ┆ 5780               │
│ MANHATTAN     ┆ 2021 ┆ 26                       ┆ 4632                      ┆ 4658               │
│ QUEENS        ┆ 2021 ┆ 34                       ┆ 8452                      ┆ 8486               │
│ STATEN ISLAND ┆ 2021 ┆ 6                        ┆ 1095                     

In [4]:
data.shape

(2125763, 31)

In [5]:
data.head()

borough,zip_code,latitude,longitude,on_street_name,cross_street_name,off_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,date,year,month,time,hour,number_of_casualty
str,i64,f64,f64,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,date,i32,i8,time,i8,i64
null,null,null,null,"""WHITESTONE EXPRESSWAY""","""20 AVENUE""",null,2,0,0,0,0,0,2,0,"""Aggressive Driving/Road Rage""","""Unspecified""",null,null,null,"""Sedan""","""Sedan""",null,null,null,2021-09-11,2021,9,02:39:00,2,2
null,null,null,null,"""QUEENSBORO BRIDGE UPPER""",null,null,1,0,0,0,0,0,1,0,"""Pavement Slippery""",null,null,null,null,"""Sedan""",null,null,null,null,2022-03-26,2022,3,11:45:00,11,1
null,null,null,null,"""THROGS NECK BRIDGE""",null,null,0,0,0,0,0,0,0,0,"""Following Too Closely""","""Unspecified""",null,null,null,"""Sedan""","""Pick-up Truck""",null,null,null,2022-06-29,2022,6,06:55:00,6,0
"""BROOKLYN""",11208,40.667202,-73.8665,null,null,"""1211 LORING AVENUE""",0,0,0,0,0,0,0,0,"""Unspecified""",null,null,null,null,"""Sedan""",null,null,null,null,2021-09-11,2021,9,09:35:00,9,0
"""BROOKLYN""",11233,40.683304,-73.917274,"""SARATOGA AVENUE""","""DECATUR STREET""",null,0,0,0,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,2021-12-14,2021,12,08:13:00,8,0
